In [1]:
import numpy as np
import dicom
import sys
sys.path.append('..')
import cv2
import os
from AlgoEngine.sts import getSTSHistogram
from AlgoEngine.similarity import getSTSEmd
from AlgoEngine.utils import getContours, getROINumber
from math import sqrt
import time
import matplotlib.pyplot as plt
start_time = time.time() # For runtime test


## Inputs to Function

Our inputs are two generated OVHs- One for the Query Patient (`UCLA_PR_5`) and one for the historical patient
(`'UCLA_PR_6'`). We currently compare OVH for a single PTV-OAR pair.

In [2]:
BASE_DIR = '/home/radiation/RadiationTherapyDecisionSupport/data/'
StudyID = 'UCLA_PR_5'
structureset = dicom.read_file(BASE_DIR + StudyID + '/structureset.dcm')
n_bins = 10

ctFilenames = [fl for fl in os.listdir(BASE_DIR + StudyID) if 'CT.' in fl]
numImages = len(ctFilenames)

sampleCTImage = dicom.read_file(BASE_DIR + StudyID + '/' + ctFilenames[0])
width = sampleCTImage.Columns
height = sampleCTImage.Rows
row_spacing = float(sampleCTImage.PixelSpacing[0])
column_spacing = float(sampleCTImage.PixelSpacing[1])
slice_spacing = float(sampleCTImage.SliceThickness)

block_shape = (width, height, numImages)
slice_position_z = np.zeros((numImages)).astype(np.float32) 

for i, fl in enumerate(ctFilenames):
    slice_position_z[i] = dicom.read_file(BASE_DIR + StudyID + '/' + fl).ImagePositionPatient[2]

In [3]:
PTV_ROI_NUM = getROINumber(structureset, 'PTV')
roiNumPlanes = len(structureset.ROIContourSequence[PTV_ROI_NUM].ContourSequence) 

contour_data = {} 
image_orientation = {} 
image_position = {} 
pixel_spacing = {} 

for index in range(0, roiNumPlanes):
    
    imageSOP = structureset.ROIContourSequence[PTV_ROI_NUM].ContourSequence[index].ContourImageSequence[0].ReferencedSOPInstanceUID
    
    planeContourData = np.array(structureset.ROIContourSequence[PTV_ROI_NUM].ContourSequence[index].ContourData)
    planeContourData = planeContourData.reshape(planeContourData.shape[0] // 3 , 3)
    
    contour_data[imageSOP] = planeContourData
    imagei = dicom.read_file(BASE_DIR + StudyID + '/CT.' + imageSOP + '.dcm')
    
    image_orientation[imageSOP] = imagei.ImageOrientationPatient
    image_position[imageSOP] = imagei.ImagePositionPatient
    pixel_spacing[imageSOP] = imagei.PixelSpacing
ptv_contour_block, ptv_roi_block = getContours(block_shape, slice_position_z, contour_data, image_orientation, image_position, pixel_spacing)

In [4]:
OAR_ROI_NUM = getROINumber(structureset, 'Bladder')
roiNumPlanes = len(structureset.ROIContourSequence[OAR_ROI_NUM].ContourSequence) 

contour_data = {}
image_orientation = {}
image_position = {} 
pixel_spacing = {}

for index in range(0, roiNumPlanes):
    
    imageSOP = structureset.ROIContourSequence[OAR_ROI_NUM].ContourSequence[index].ContourImageSequence[0].ReferencedSOPInstanceUID
    planeContourData = np.array(structureset.ROIContourSequence[OAR_ROI_NUM].ContourSequence[index].ContourData)
    planeContourData = planeContourData.reshape(planeContourData.shape[0] // 3 , 3)
    
    contour_data[imageSOP] = planeContourData
    imagei = dicom.read_file(BASE_DIR + StudyID + '/CT.' + imageSOP + '.dcm')
    
    image_orientation[imageSOP] = imagei.ImageOrientationPatient
    image_position[imageSOP] = imagei.ImagePositionPatient
    pixel_spacing[imageSOP] = imagei.PixelSpacing
_, oar_roi_block = getContours(block_shape, slice_position_z, contour_data, image_orientation, image_position, pixel_spacing)

In [5]:
_, _, _, query_sts = getSTSHistogram(ptv_roi_block, oar_roi_block, n_bins)

In [6]:
StudyID = 'UCLA_PR_6'
structureset = dicom.read_file(BASE_DIR + StudyID + '/structureset.dcm')

ctFilenames = [fl for fl in os.listdir(BASE_DIR + StudyID) if 'CT.' in fl]
numImages = len(ctFilenames)

sampleCTImage = dicom.read_file(BASE_DIR + StudyID + '/' + ctFilenames[0])
width = sampleCTImage.Columns
height = sampleCTImage.Rows
row_spacing = float(sampleCTImage.PixelSpacing[0])
column_spacing = float(sampleCTImage.PixelSpacing[1])
slice_spacing = float(sampleCTImage.SliceThickness)

block_shape = (width, height, numImages)
slice_position_z = np.zeros((numImages)).astype(np.float32) 

for i, fl in enumerate(ctFilenames):
    slice_position_z[i] = dicom.read_file(BASE_DIR + StudyID + '/' + fl).ImagePositionPatient[2]

In [7]:
PTV_ROI_NUM = getROINumber(structureset, 'PTV')
roiNumPlanes = len(structureset.ROIContourSequence[PTV_ROI_NUM].ContourSequence) 

contour_data = {} 
image_orientation = {} 
image_position = {} 
pixel_spacing = {} 

for index in range(0, roiNumPlanes):
    
    imageSOP = structureset.ROIContourSequence[PTV_ROI_NUM].ContourSequence[index].ContourImageSequence[0].ReferencedSOPInstanceUID
    
    planeContourData = np.array(structureset.ROIContourSequence[PTV_ROI_NUM].ContourSequence[index].ContourData)
    planeContourData = planeContourData.reshape(planeContourData.shape[0] // 3 , 3)
    
    contour_data[imageSOP] = planeContourData
    imagei = dicom.read_file(BASE_DIR + StudyID + '/CT.' + imageSOP + '.dcm')
    
    image_orientation[imageSOP] = imagei.ImageOrientationPatient
    image_position[imageSOP] = imagei.ImagePositionPatient
    pixel_spacing[imageSOP] = imagei.PixelSpacing
ptv_contour_block, ptv_roi_block = getContours(block_shape, slice_position_z, contour_data, image_orientation, image_position, pixel_spacing)

In [8]:
OAR_ROI_NUM = getROINumber(structureset, 'Bladder')
roiNumPlanes = len(structureset.ROIContourSequence[OAR_ROI_NUM].ContourSequence) 

contour_data = {}
image_orientation = {}
image_position = {} 
pixel_spacing = {}

for index in range(0, roiNumPlanes):
    
    imageSOP = structureset.ROIContourSequence[OAR_ROI_NUM].ContourSequence[index].ContourImageSequence[0].ReferencedSOPInstanceUID
    planeContourData = np.array(structureset.ROIContourSequence[OAR_ROI_NUM].ContourSequence[index].ContourData)
    planeContourData = planeContourData.reshape(planeContourData.shape[0] // 3 , 3)
    
    contour_data[imageSOP] = planeContourData
    imagei = dicom.read_file(BASE_DIR + StudyID + '/CT.' + imageSOP + '.dcm')
    
    image_orientation[imageSOP] = imagei.ImageOrientationPatient
    image_position[imageSOP] = imagei.ImagePositionPatient
    pixel_spacing[imageSOP] = imagei.PixelSpacing
_, oar_roi_block = getContours(block_shape, slice_position_z, contour_data, image_orientation, image_position, pixel_spacing)

In [9]:
_, _, _, historical_sts = getSTSHistogram(ptv_roi_block, oar_roi_block, n_bins)

## Function Here

In [10]:
emd  = getSTSEmd(query_sts, historical_sts)

## Testing

We rely exclusively on OpenCV for accurate EMD implementation- we have minimal way to confirm if this is the correct EMD or if the dissimilarity is correct. So we just print the EMD value and some rudimentary tests:

1) The EMD of two identical histograms should be 0
2) Runtime

In [ ]:
print(emd)

8.17900562286377


In [ ]:
print( getSTSEmd(query_sts, query_sts) == 0)
print( getSTSEmd(historical_sts, historical_sts) == 0)

True


In [ ]:
print("Runtime (in minutes + fraction) of functions (single PTV and single OAR): " + str((time.time() - start_time) / 60))